In [1]:
import numpy as np
import pandas as pd
%matplotlib notebook

In [2]:
n = str(2)
fullDF = pd.read_csv("data/Chiller"+n+"_full.csv")
num_samples = fullDF.shape[0]
window_size = 60

In [3]:
fullDF.head()

,ts,ch1Watt,ch2Watt,ch3Watt,value1,value2,value3,value4,conflowRate,conflowSpeed,evaflowRate,evaflowSpeed
0,2017-05-01 00:00:00,325,0,324,18.580934,21.582337,23.633118,23.451539,0.0,0.0,0.0,0.0
1,2017-05-01 00:01:00,326,0,328,18.602295,21.539612,23.558351,23.398133,0.0,0.0,0.0,0.0
2,2017-05-01 00:02:00,327,0,324,18.495484,21.614379,23.483581,23.323363,0.0,0.0,0.0,0.0
3,2017-05-01 00:03:00,338,0,325,18.474123,21.571657,23.601076,23.205871,0.0,0.0,0.0,0.0
4,2017-05-01 00:04:00,334,0,328,18.559570,21.560973,23.536987,23.227236,0.0,0.0,0.0,0.0


In [4]:
Watt = fullDF[["ch1Watt", "ch2Watt", "ch3Watt"]].as_matrix()
Value = fullDF[["value1", "value2", "value3", "value4"]].as_matrix()
Conflow = fullDF[["conflowRate", "conflowSpeed"]].as_matrix()
Evaflow = fullDF[["evaflowRate", "evaflowSpeed"]].as_matrix()
Total_Watt = np.sum(Watt, axis = 1)

In [5]:
from sklearn import preprocessing

In [6]:
Watt_scaled = preprocessing.scale(Watt)
Value_scaled = preprocessing.scale(Value)
Conflow_scaled = preprocessing.scale(Conflow)
Evaflow_scaled = preprocessing.scale(Evaflow)
Total_Watt_scaled = preprocessing.scale(Total_Watt)

/Users/duynguyen/anaconda/envs/python2/lib/python2.7/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, _DataConversionWarning)


In [7]:
Hour = pd.to_datetime(fullDF.ts).dt.hour.as_matrix()
Hour_transformed = np.zeros([num_samples,24])
for i in range(num_samples):
    Hour_transformed[i, Hour[i]] = 1

In [8]:
Day = pd.to_datetime(fullDF.ts).dt.day.as_matrix()
Day_transformed = np.zeros([num_samples,31])
for i in range(num_samples):
    Day_transformed[i, Day[i] - 1] = 1

In [9]:
Month = pd.to_datetime(fullDF.ts).dt.month.as_matrix()
Month_transformed = np.zeros([num_samples,12])
for i in range(num_samples):
    Month_transformed[i, Month[i] - 1] = 1

In [14]:
# Data = np.concatenate([Hour_transformed, Value_scaled, Conflow_scaled, Evaflow_scaled], axis =1)
Data = np.concatenate([Value_scaled, Conflow_scaled, Evaflow_scaled], axis =1)

In [15]:
dimension = Data.shape[1]

In [16]:
Data_align = np.zeros((window_size, num_samples - window_size + 1, dimension))
for i in range(0, window_size):
    Data_align[i] = Data[i: num_samples - window_size + 1 + i]

In [17]:
Full_Data = Data_align.transpose((1,0,2))

In [18]:
Power = Total_Watt[:num_samples - window_size + 1].reshape((-1,1))

In [27]:
Moment = np.concatenate([np.zeros((1,1)),Power[1:] - Power[:-1]], axis = 0)

In [29]:
Force = np.concatenate([np.zeros((1,1)),Moment[1:] - Moment[:-1]], axis = 0)

In [36]:
import keras
from keras import layers
from keras.layers import Lambda, Activation,recurrent, Bidirectional, Dense, Flatten, Conv1D, Dropout, LSTM, GRU, concatenate, multiply, add, Reshape, MaxPooling1D, BatchNormalization
from keras.models import Model, load_model
import keras.backend as K

Using TensorFlow backend.


In [79]:
def mean_pred(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true)/(y_true + 0.01))

In [80]:
inp = layers.Input(shape=(window_size, dimension), dtype=Full_Data.dtype)

In [81]:
flatten = Flatten()(inp)
dense1 = Dense(500, activation='relu')(flatten)
dense2 = Dense(500, activation='relu')(dense1)
dense3 = Dense(500, activation='relu')(dense2)

In [82]:
output1 = Dense(1)(dense3)
output2 = Dense(1)(dense3)
output3 = Dense(1)(dense3)

In [86]:
model = Model([inp],[output1,output2,output3])
model.compile(optimizer='adam',
              loss='mean_absolute_error', metrics=[mean_pred])

In [87]:
test_inp = Full_Data[:380000]
test_out1 = Power[:380000]
test_out2 = Moment[:380000]
test_out3 = Force[:380000]
val_inp = Full_Data[380000:]
val_out1 = Power[380000:]
val_out2 = Moment[380000:]
val_out3 = Force[380000:]

In [88]:
model.fit([test_inp], [test_out1, test_out2, test_out3],
          epochs=5,
          validation_data=([val_inp], [val_out1,val_out2,val_out3]))

Train on 380000 samples, validate on 50511 samples
Epoch 1/5
380000/380000 [==============================] - 189s - loss: 512553.0309 - dense_22_loss: 182295.8131 - dense_23_loss: 151287.8207 - dense_24_loss: 178969.3966 - dense_22_mean_pred: 9.9536 - dense_23_mean_pred: 9327553.6775 - dense_24_mean_pred: 10249595.9540 - val_loss: 1313.4491 - val_dense_22_loss: 453.4910 - val_dense_23_loss: 312.3053 - val_dense_24_loss: 547.6529 - val_dense_22_mean_pred: 0.0832 - val_dense_23_mean_pred: 79.0115 - val_dense_24_mean_pred: 107.7628
Epoch 2/5
380000/380000 [==============================] - 180s - loss: 5781.0792 - dense_22_loss: 3473.0483 - dense_23_loss: 887.7219 - dense_24_loss: 1420.3090 - dense_22_mean_pred: 0.5370 - dense_23_mean_pred: 7593.3147 - dense_24_mean_pred: 8290.9825 - val_loss: 1293.9322 - val_dense_22_loss: 430.9985 - val_dense_23_loss: 310.1828 - val_dense_24_loss: 552.7508 - val_dense_22_mean_pred: 0.0671 - val_dense_23_mean_pred: 116.0088 - val_dense_24_mean_pred: 277

In [89]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_4 (InputLayer)             (None, 60, 8)         0                                            
____________________________________________________________________________________________________
flatten_4 (Flatten)              (None, 480)           0           input_4[0][0]                    
____________________________________________________________________________________________________
dense_19 (Dense)                 (None, 500)           240500      flatten_4[0][0]                  
____________________________________________________________________________________________________
dense_20 (Dense)                 (None, 500)           250500      dense_19[0][0]                   
___________________________________________________________________________________________

In [90]:
pred = model.predict(val_inp, verbose=1)

49856/50511 [============================>.] - ETA: 0s

In [91]:
pred

[array([[ 662.85998535],
        [ 662.8838501 ],
        [ 663.18438721],
        ..., 
        [ 652.06488037],
        [ 651.63739014],
        [ 651.55389404]], dtype=float32), array([[-0.41907763],
        [-0.42519069],
        [-0.43164992],
        ..., 
        [-0.09275271],
        [-0.08329464],
        [-0.07088925]], dtype=float32), array([[-0.39362285],
        [-0.4010348 ],
        [-0.41020533],
        ..., 
        [ 0.03292421],
        [ 0.04360536],
        [ 0.04971173]], dtype=float32)]

In [92]:
a = (pred[0] - val_out1)/val_out1

In [97]:
b = a > 1

In [98]:
np.sum(b)

31

In [99]:
Full_Data.shape

(430511, 60, 8)